In [1]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
from salishsea_tools import geo_tools, nc_tools, tidetools, viz_tools
import xarray as xr
from IPython.core.display import display, HTML
import scipy.signal as signal
import glob
import matplotlib.cm as cm
import datetime
display(HTML("<style>.container { width:90% !important; }</style>"))

%matplotlib inline

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
winds_data = xr.open_dataset(
    'https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSaSurfaceAtmosphereFieldsV1')

In [3]:
winds_geo = xr.open_dataset(
    'https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSaAtmosphereGridV1')

In [14]:
grid = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSnBathymetryV17-02')

In [21]:
tracers = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSg3DTracerFields1hV18-12')

In [23]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [35]:
ssh = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSgSurfaceTracerFields1hV18-12')

In [38]:
winds = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSaSurfaceAtmosphereFieldsV1')

In [75]:
def make_stats(time_slce, location, graph = False):
    if location == 'salmon':
        ss_coords = [256,268]
        wind_coords = [99,159]
    if location == 'turn':
        ss_coords = [343,250]
        wind_coords = [116,150]
    if location == 'sog':
        ss_coords = [474,252]
        wind_coords = [143,141]
    sst_ts = (
    tracers.salinity
    .sel(depth=0.5, method='nearest')
    .isel(gridY=ss_coords[0], gridX=ss_coords[1])
    .sel(time=time_slice)
    )
    ssh_ts = (
    ssh.ssh
    .isel(gridY=ss_coords[0], gridX=ss_coords[1])
    .sel(time=time_slice)
    )
    u_winds = (
    winds.u_wind
    .isel(gridY=wind_coords[0], gridX=wind_coords[1])
    .sel(time=time_slice)
    )
    v_winds = (
    winds.v_wind
    .isel(gridY=wind_coords[0], gridX=wind_coords[1])
    .sel(time=time_slice)
    )
    wind_speed = np.sqrt(u_winds**2 + v_winds**2)
    
    print('surface salinity mean = ' + str(sst_ts.values.mean()))
    print('surface salinity min = ' + str(sst_ts.values.min()))
    print('surface salinity max = ' + str(sst_ts.values.max()))
    print('surface salinity std = ' + str(sst_ts.values.std()))
    
    print('ssh mean = ' + str(ssh_ts.values.mean()))
    print('ssh min = ' + str(ssh_ts.values.min()))
    print('ssh max = ' + str(ssh_ts.values.max()))
    print('ssh std = ' + str(ssh_ts.values.std()))
    
    print('wind speed mean = ' + str(wind_speed.values.mean()))
    print('wind speed min = ' + str(wind_speed.values.min()))
    print('wind speed max = ' + str(wind_speed.values.max()))
    print('wind speed std = ' + str(wind_speed.values.std()))
    
    if graph == True:
        fig, ax = plt.subplots(3,1,figsize=(18, 10))
        sst_ts.plot(ax=ax[0])
        ax[0].grid(axis='x')
        ax[0].set_title('Surface Salinity')
        ax[0].set_xticklabels([])

        ssh_ts.plot(ax=ax[1])
        ax[1].grid(axis='x')
        ax[1].set_title('Sea Surface Height')
        ax[1].set_xticklabels([])

        wind_speed.plot(ax=ax[2])
        ax[2].grid(axis='x')
        ax[2].set_title('Wind Speed')

In [ ]:
make_stats(slice('2017-08-01 00:00:00', '2017-08-09 00:00:00'), 'salmon', graph = True)